In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\Real_projects\\End_to_end_ml_project_mlops\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Dell\\OneDrive\\Desktop\\Real_projects\\End_to_end_ml_project_mlops'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URl : str
    local_data_file : Path
    unzip_dir : Path

In [9]:
from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml,create_directories

In [13]:
class ConfigurationManager:
    def __init__(self,
                 config_filapath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filapath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URl= config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [14]:
import os
import urllib.request as request
import zipfile
from src.mlproject import logger
from src.mlproject.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config : DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url = self.config.source_URl,
                filename = self.config.local_data_file
            )
            logger.info("{} download ! with following info {}:".format(filename,headers))
        else:
            logger.info("file already exists of size : {}".format(get_size(Path(self.config.local_data_file))))
            
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            
            zip_ref.extractall(unzip_path)
            

Creating pipeline

In [20]:
try:
    
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2023-09-13 16:03:19,011 - common - INFO - yaml file : {} loaded successfully
2023-09-13 16:03:19,013 - common - INFO - yaml file : {} loaded successfully
2023-09-13 16:03:19,015 - common - INFO - yaml file : {} loaded successfully
2023-09-13 16:03:19,017 - common - INFO - Created directory : artifacts
2023-09-13 16:03:19,019 - common - INFO - Created directory : artifacts/data_ingestion
2023-09-13 16:03:20,872 - 675423121 - INFO - artifacts/data_ingestion/data.zip download ! with following info Connection: close
Content-Length: 73146
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "6eaa95417eb9f251f784f17f8560e1d90a6ff25c2f576ef85f1744c99593fa06"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 51B8:330FB:194754:23E8C6:65018FEF
Accept-Ranges: bytes
Date: Wed, 13 Sep 2023 10:33:20 GMT
Via: 1.1